### dev_notebook_

This is your development environment...you may start:
- Exploring your data.
- Testing the ad-hoc methods.
- Designing your pipeline.

In [4]:
#imports

import pandas as pd 
import requests
import numpy as np

In [5]:
bicimad = pd.read_csv("../data/raw/bicimad_stations.csv", sep="\t")
bicimad

,Unnamed: 0,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry.type,geometry.coordinates
0,0,1,1a - Puerta del Sol A,3,1a,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.7018341, 40.4172137]"
1,1,2,1b - Puerta del Sol B,3,1b,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.701602938060457, 40.41731271011562]"
2,2,3,2 - Miguel Moya,3,2,Calle Miguel Moya nº 1,1,1,24,0,0,0,Point,"[-3.7058415, 40.4205886]"
3,3,4,3 - Plaza Conde Suchil,2,3,Plaza del Conde del Valle de Súchil nº 3,1,0,18,9,9,0,Point,"[-3.7069171, 40.4302937]"
4,4,5,4 - Malasaña,1,4,Calle Manuela Malasaña nº 5,1,0,24,23,1,0,Point,"[-3.7025875, 40.4285524]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,259,265,257 - INEF,0,257,Avenida Juan de Herrera frente a la calle Paul...,1,0,24,3,19,0,Point,"[-3.72997, 40.43896]"
260,260,266,258 - Ciudad Universitaria 1,2,258,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,9,14,0,Point,"[-3.72699, 40.44375]"
261,261,267,259 - Ciudad Universitaria 2,2,259,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,12,10,0,Point,"[-3.72693, 40.44342]"
262,262,268,260 - Facultad Biología,2,260,Calle José Antonio Novais frente al nº 12,1,0,24,12,10,0,Point,"[-3.7272945, 40.4483322]"


In [6]:
bicimad = bicimad.drop(["Unnamed: 0"], axis=1)

In [7]:
bicimad

,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry.type,geometry.coordinates
0,1,1a - Puerta del Sol A,3,1a,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.7018341, 40.4172137]"
1,2,1b - Puerta del Sol B,3,1b,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.701602938060457, 40.41731271011562]"
2,3,2 - Miguel Moya,3,2,Calle Miguel Moya nº 1,1,1,24,0,0,0,Point,"[-3.7058415, 40.4205886]"
3,4,3 - Plaza Conde Suchil,2,3,Plaza del Conde del Valle de Súchil nº 3,1,0,18,9,9,0,Point,"[-3.7069171, 40.4302937]"
4,5,4 - Malasaña,1,4,Calle Manuela Malasaña nº 5,1,0,24,23,1,0,Point,"[-3.7025875, 40.4285524]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,265,257 - INEF,0,257,Avenida Juan de Herrera frente a la calle Paul...,1,0,24,3,19,0,Point,"[-3.72997, 40.43896]"
260,266,258 - Ciudad Universitaria 1,2,258,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,9,14,0,Point,"[-3.72699, 40.44375]"
261,267,259 - Ciudad Universitaria 2,2,259,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,12,10,0,Point,"[-3.72693, 40.44342]"
262,268,260 - Facultad Biología,2,260,Calle José Antonio Novais frente al nº 12,1,0,24,12,10,0,Point,"[-3.7272945, 40.4483322]"


In [8]:
split_bicimad = bicimad['geometry.coordinates'].str.strip('[]').str.split(',', expand=True).astype('float64')
split_bicimad.columns = ['longitude', 'latitude']
bicimad= pd.concat([bicimad,split_bicimad],axis=1)
bicimad

,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry.type,geometry.coordinates,longitude,latitude
0,1,1a - Puerta del Sol A,3,1a,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.7018341, 40.4172137]",-3.701834,40.417214
1,2,1b - Puerta del Sol B,3,1b,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.701602938060457, 40.41731271011562]",-3.701603,40.417313
2,3,2 - Miguel Moya,3,2,Calle Miguel Moya nº 1,1,1,24,0,0,0,Point,"[-3.7058415, 40.4205886]",-3.705842,40.420589
3,4,3 - Plaza Conde Suchil,2,3,Plaza del Conde del Valle de Súchil nº 3,1,0,18,9,9,0,Point,"[-3.7069171, 40.4302937]",-3.706917,40.430294
4,5,4 - Malasaña,1,4,Calle Manuela Malasaña nº 5,1,0,24,23,1,0,Point,"[-3.7025875, 40.4285524]",-3.702587,40.428552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,265,257 - INEF,0,257,Avenida Juan de Herrera frente a la calle Paul...,1,0,24,3,19,0,Point,"[-3.72997, 40.43896]",-3.729970,40.438960
260,266,258 - Ciudad Universitaria 1,2,258,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,9,14,0,Point,"[-3.72699, 40.44375]",-3.726990,40.443750
261,267,259 - Ciudad Universitaria 2,2,259,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,12,10,0,Point,"[-3.72693, 40.44342]",-3.726930,40.443420
262,268,260 - Facultad Biología,2,260,Calle José Antonio Novais frente al nº 12,1,0,24,12,10,0,Point,"[-3.7272945, 40.4483322]",-3.727295,40.448332


In [9]:
bicimad = bicimad.drop(["geometry.coordinates"], axis = 1)

In [10]:
bicimad = bicimad.drop(["light", "number", "activate", "no_available", "geometry.type"], axis = 1)

In [11]:
bicimad

,id,name,address,total_bases,dock_bikes,free_bases,reservations_count,longitude,latitude
0,1,1a - Puerta del Sol A,Puerta del Sol nº 1,30,0,0,0,-3.701834,40.417214
1,2,1b - Puerta del Sol B,Puerta del Sol nº 1,30,0,0,0,-3.701603,40.417313
2,3,2 - Miguel Moya,Calle Miguel Moya nº 1,24,0,0,0,-3.705842,40.420589
3,4,3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,18,9,9,0,-3.706917,40.430294
4,5,4 - Malasaña,Calle Manuela Malasaña nº 5,24,23,1,0,-3.702587,40.428552
...,...,...,...,...,...,...,...,...,...
259,265,257 - INEF,Avenida Juan de Herrera frente a la calle Paul...,24,3,19,0,-3.729970,40.438960
260,266,258 - Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,24,9,14,0,-3.726990,40.443750
261,267,259 - Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,24,12,10,0,-3.726930,40.443420
262,268,260 - Facultad Biología,Calle José Antonio Novais frente al nº 12,24,12,10,0,-3.727295,40.448332


In [12]:
bicipark = pd.read_csv("../data/raw/bicipark_stations.csv", sep=";")
bicipark

,Unnamed: 0,stationId,stationName,address,city,zip_code,total_places,free_places,enabled,reserved_places,geometry.type,geometry.coordinates
0,0,74,Bicipark Fuente de la Mora,Estación de Fuente de la Mora,Madrid,28050,10,9,1,0,Point,"[-3.6630679, 40.4846838]"
1,1,22,Bicipark Orense,"Avenida General Perón, 27",Madrid,28020,15,9,1,1,Point,"[-3.693897, 40.452973]"
2,2,49,Bicipark Montalban,"Calle de Montalbán, 5",Madrid,28014,16,15,1,0,Point,"[-3.690786, 40.418146]"
3,3,12,Bicipark Almagro,"Calle Almagro, 11",Madrid,28010,16,12,1,0,Point,"[-3.692879, 40.430361]"
4,4,9,Bicipark Recoletos,"Paseo de Recoletos, 2",Madrid,28001,10,3,0,0,Point,"[-3.692245, 40.420354]"
5,5,5,Bicipark Recuerdo,"Calle Hiedra, 26",Madrid,28036,13,13,1,0,Point,"[-3.678921, 40.472419]"
6,6,25,Bicipark Salamanca,Pl. del Marqués de Salamanca,Madrid,28006,5,-1,1,0,Point,"[-3.679172, 40.430315]"
7,7,67,Bicipark Villa de Paris,"Calle de Orellana, 23, 21",Madrid,28004,10,8,1,1,Point,"[-3.693698, 40.425608]"
8,8,1154,Bicipark Pitis,"Calle Pau Arroyo del Fresno, 181",Madrid,28049,10,7,1,0,Point,"[-3.7249142, 40.4959425]"
9,9,1169,Bicipark Aviación,Parking Aviación Española,Madrid,28024,10,10,1,0,Point,"[-3.78369522, 40.383491516]"


In [13]:
bicipark = bicipark.drop(["Unnamed: 0"], axis=1)

In [14]:
bicipark = bicipark.drop(["zip_code", "enabled", "reserved_places","geometry.type"], axis = 1)

In [15]:
split_bicipark = bicipark['geometry.coordinates'].str.strip('[]').str.split(',', expand=True).astype('float64')
split_bicipark.columns = ['longitude', 'latitude']
bicipark= pd.concat([bicipark,split_bicipark],axis=1)
bicipark

,stationId,stationName,address,city,total_places,free_places,geometry.coordinates,longitude,latitude
0,74,Bicipark Fuente de la Mora,Estación de Fuente de la Mora,Madrid,10,9,"[-3.6630679, 40.4846838]",-3.663068,40.484684
1,22,Bicipark Orense,"Avenida General Perón, 27",Madrid,15,9,"[-3.693897, 40.452973]",-3.693897,40.452973
2,49,Bicipark Montalban,"Calle de Montalbán, 5",Madrid,16,15,"[-3.690786, 40.418146]",-3.690786,40.418146
3,12,Bicipark Almagro,"Calle Almagro, 11",Madrid,16,12,"[-3.692879, 40.430361]",-3.692879,40.430361
4,9,Bicipark Recoletos,"Paseo de Recoletos, 2",Madrid,10,3,"[-3.692245, 40.420354]",-3.692245,40.420354
5,5,Bicipark Recuerdo,"Calle Hiedra, 26",Madrid,13,13,"[-3.678921, 40.472419]",-3.678921,40.472419
6,25,Bicipark Salamanca,Pl. del Marqués de Salamanca,Madrid,5,-1,"[-3.679172, 40.430315]",-3.679172,40.430315
7,67,Bicipark Villa de Paris,"Calle de Orellana, 23, 21",Madrid,10,8,"[-3.693698, 40.425608]",-3.693698,40.425608
8,1154,Bicipark Pitis,"Calle Pau Arroyo del Fresno, 181",Madrid,10,7,"[-3.7249142, 40.4959425]",-3.724914,40.495942
9,1169,Bicipark Aviación,Parking Aviación Española,Madrid,10,10,"[-3.78369522, 40.383491516]",-3.783695,40.383492


In [16]:
bicipark = bicipark.drop(["geometry.coordinates"], axis= 1)

In [17]:
bicipark

,stationId,stationName,address,city,total_places,free_places,longitude,latitude
0,74,Bicipark Fuente de la Mora,Estación de Fuente de la Mora,Madrid,10,9,-3.663068,40.484684
1,22,Bicipark Orense,"Avenida General Perón, 27",Madrid,15,9,-3.693897,40.452973
2,49,Bicipark Montalban,"Calle de Montalbán, 5",Madrid,16,15,-3.690786,40.418146
3,12,Bicipark Almagro,"Calle Almagro, 11",Madrid,16,12,-3.692879,40.430361
4,9,Bicipark Recoletos,"Paseo de Recoletos, 2",Madrid,10,3,-3.692245,40.420354
5,5,Bicipark Recuerdo,"Calle Hiedra, 26",Madrid,13,13,-3.678921,40.472419
6,25,Bicipark Salamanca,Pl. del Marqués de Salamanca,Madrid,5,-1,-3.679172,40.430315
7,67,Bicipark Villa de Paris,"Calle de Orellana, 23, 21",Madrid,10,8,-3.693698,40.425608
8,1154,Bicipark Pitis,"Calle Pau Arroyo del Fresno, 181",Madrid,10,7,-3.724914,40.495942
9,1169,Bicipark Aviación,Parking Aviación Española,Madrid,10,10,-3.783695,40.383492


In [18]:
places = requests.get("https://datos.madrid.es/egob/catalogo/300614-0-centros-educativos.json")

In [19]:
places = places.json()

In [20]:
places.keys()

dict_keys(['@context', '@graph'])

In [21]:
places = places["@graph"]

In [22]:
places = pd.json_normalize(places)

In [23]:
places = places.drop(["@id", "id", "relation", "address.district.@id", "address.area.@id", "address.locality", "address.postal-code", "organization.organization-desc", "organization.accesibility", "organization.schedule", "organization.services", "@type"], axis = 1)

In [24]:
places

,title,address.street-address,location.latitude,location.longitude,organization.organization-name
0,ABC Schule (Alemán),CALLE LEIZARAN 27,40.447932,-3.681382,ABC Schule (Alemán)
1,Academia Darío Estudio,CALLE GRAN VIA 63,40.422491,-3.709818,Academia Darío Estudio
2,AFI Escuela de Finanzas,CALLE MARQUES DE VILLAMEJOR 5,40.431247,-3.687559,AFI Escuela de Finanzas
3,Agencia Estatal Consejo Superior de Investigac...,CALLE SERRANO 115 BIS,40.441165,-3.687752,Agencia Estatal Consejo Superior de Investigac...
4,Alliance Française de Madrid,CUESTA SANTO DOMINGO 13,40.420151,-3.708855,Alliance Française de Madrid
...,...,...,...,...,...
1879,Universidad Pontificia Comillas. Facultad de D...,CALLE ALBERTO AGUILERA 23,40.429980,-3.711258,Universidad Pontificia Comillas. Facultad de D...
1880,Universidad Pontificia Comillas. Facultad de T...,CALLE UNIVERSIDAD DE COMILLAS 3 y 5,40.552126,-3.682802,Universidad Pontificia Comillas. Facultad de T...
1881,Universidad Pontificia de Comillas. Facultad d...,CALLE ALBERTO AGUILERA 23,40.429980,-3.711258,Universidad Pontificia de Comillas. Facultad d...
1882,Universidad Pontificia de Comillas. Facultad d...,CALLE ALBERTO AGUILERA 23,40.429980,-3.711258,Universidad Pontificia de Comillas. Facultad d...


In [31]:
"""
La fórmula proporcionada no es eficiente. Llevo horas corriendo el código y aún no me ha dado resultado. 
He descubierto que vectorizando las operaciones (haciéndolas directamente sobre matrices, 
en lugar de iterando de fila en fila) da un resultado inmediato. 
"""
def nearest_bicimad():
    data_list = []
    
    #Convierto la latitud y la longitud a radianes para realizar las operaciones (en la documentación se
    #menciona que es necesario). Luego los convierto a numpy porque si no da error.

    places_lat_rad = np.radians(places['location.latitude'].to_numpy())
    places_lon_rad = np.radians(places['location.longitude'].to_numpy())
    bicimad_lat_rad = np.radians(bicimad['latitude'].to_numpy())
    bicimad_lon_rad = np.radians(bicimad['longitude'].to_numpy())

    dlat = bicimad_lat_rad[:, np.newaxis] - places_lat_rad
    dlon = bicimad_lon_rad[:, np.newaxis] - places_lon_rad

    #Fórmula de Haversine para calcular las distancias. 
    
    a = np.sin(dlat / 2) ** 2 + np.cos(bicimad_lat_rad[:, np.newaxis]) * np.cos(places_lat_rad) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    distance_matrix = c * 6371000 


    min_distance_indices = np.argmin(distance_matrix, axis=0)

    for x in range(len(places["title"])):
        station_index = min_distance_indices[x]
        station = bicimad['name'].iloc[station_index].split('- ')[1]
        station_address = bicimad["address"].iloc[station_index]
        bikes_available = bicimad["dock_bikes"].iloc[station_index]
        place_address = places["address.street-address"][x]
        place = places["title"][x]
        min_distance = round(distance_matrix[station_index, x], 2)
        data_list.append({"place": place, "place_address": place_address, "station_name": station, "bikes_available": bikes_available, "station_address": station_address,  "distance": min_distance})
        
    return pd.DataFrame(data_list)

In [32]:
nearest_bicimad()

,place,place_address,station_name,bikes_available,station_address,distance
0,ABC Schule (Alemán),CALLE LEIZARAN 27,Doctor Arce 45,1,Avenida del Doctor Arce nº 45,146.60
1,Academia Darío Estudio,CALLE GRAN VIA 63,Plaza de España B,0,Calle Princesa nº 5,145.70
2,AFI Escuela de Finanzas,CALLE MARQUES DE VILLAMEJOR 5,Ortega y Gasset,0,Calle José Ortega y Gasset nº 4,130.77
3,Agencia Estatal Consejo Superior de Investigac...,CALLE SERRANO 115 BIS,Serrano 113,8,Calle Serrano nº 113B,141.14
4,Alliance Française de Madrid,CUESTA SANTO DOMINGO 13,Santo Domingo,23,Plaza de Santo Domingo nº 1,63.54
...,...,...,...,...,...,...
1879,Universidad Pontificia Comillas. Facultad de D...,CALLE ALBERTO AGUILERA 23,Santa Cruz del Marcenado,17,Calle Santa Cruz del Marcenado nº 24,124.87
1880,Universidad Pontificia Comillas. Facultad de T...,CALLE UNIVERSIDAD DE COMILLAS 3 y 5,Pedro Rico,0,Calle Pedro Rico nº 4,7913.76
1881,Universidad Pontificia de Comillas. Facultad d...,CALLE ALBERTO AGUILERA 23,Santa Cruz del Marcenado,17,Calle Santa Cruz del Marcenado nº 24,124.87
1882,Universidad Pontificia de Comillas. Facultad d...,CALLE ALBERTO AGUILERA 23,Santa Cruz del Marcenado,17,Calle Santa Cruz del Marcenado nº 24,124.87


In [29]:
def nearest_bicipark():
    data_list_bicipark = []
    
    #Convierto la latitud y la longitud a radianes para realizar las operaciones (en la documentación se
    #menciona que es necesario). Luego los convierto a numpy porque si no da error.

    places_lat_rad = np.radians(places['location.latitude'].to_numpy())
    places_lon_rad = np.radians(places['location.longitude'].to_numpy())
    bicipark_lat_rad = np.radians(bicipark['latitude'].to_numpy())
    bicipark_lon_rad = np.radians(bicipark['longitude'].to_numpy())

    dlat = bicipark_lat_rad[:, np.newaxis] - places_lat_rad
    dlon = bicipark_lon_rad[:, np.newaxis] - places_lon_rad

    #Fórmula de Haversine para calcular las distancias. 
    
    a = np.sin(dlat / 2) ** 2 + np.cos(bicipark_lat_rad[:, np.newaxis]) * np.cos(places_lat_rad) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    distance_matrix = c * 6371000 


    min_distance_indices = np.argmin(distance_matrix, axis=0)

    for x in range(len(places["title"])):
        station_index = min_distance_indices[x]
        station = bicipark['stationName'].iloc[station_index]
        station_address = bicipark["address"].iloc[station_index]
        spots_available = bicipark["free_places"].iloc[station_index]
        place_address = places["address.street-address"][x]
        place = places["title"][x]
        min_distance = round(distance_matrix[station_index, x], 2)
        data_list_bicipark.append({"place": place, "place_address": place_address, "station_name": station, "spots_available" : spots_available, "station_address": station_address,  "distance": min_distance})
        
    return pd.DataFrame(data_list_bicipark)

In [30]:
nearest_bicipark()

,place,place_address,station_name,spots_available,station_address,distance
0,ABC Schule (Alemán),CALLE LEIZARAN 27,Bicipark Orense,9,"Avenida General Perón, 27",1198.13
1,Academia Darío Estudio,CALLE GRAN VIA 63,Bicipark Canalejas,10,-,1020.46
2,AFI Escuela de Finanzas,CALLE MARQUES DE VILLAMEJOR 5,Bicipark Almagro,12,"Calle Almagro, 11",460.95
3,Agencia Estatal Consejo Superior de Investigac...,CALLE SERRANO 115 BIS,Bicipark Almagro,12,"Calle Almagro, 11",1277.26
4,Alliance Française de Madrid,CUESTA SANTO DOMINGO 13,Bicipark Canalejas,10,-,837.67
...,...,...,...,...,...,...
1879,Universidad Pontificia Comillas. Facultad de D...,CALLE ALBERTO AGUILERA 23,Bicipark Almagro,12,"Calle Almagro, 11",1556.24
1880,Universidad Pontificia Comillas. Facultad de T...,CALLE UNIVERSIDAD DE COMILLAS 3 y 5,Bicipark Pitis,7,"Calle Pau Arroyo del Fresno, 181",7190.17
1881,Universidad Pontificia de Comillas. Facultad d...,CALLE ALBERTO AGUILERA 23,Bicipark Almagro,12,"Calle Almagro, 11",1556.24
1882,Universidad Pontificia de Comillas. Facultad d...,CALLE ALBERTO AGUILERA 23,Bicipark Almagro,12,"Calle Almagro, 11",1556.24
